# Preceptron Learning Algorith: A Gentle Introduction

The preceptron learning algorithm has 2 parts: predicting and traning. Here we will look at both parts. 

## Predicting with PLA
Lets say we have some data formatted like this: `[(x1,y1), (x2,y2), ... (xn,yn)]`. You are given a new point `x` and our job is to predict `y` and we are told that this data fits a linear model. 

We know the equation of a line is $y=mx+b$ and so we try to figure out a good slope ($m$) and a good y-intercept ($b$). The preceptron algorithm with take the input to the system `x` and give you an output `y`. Our job now is to figure out how the input is manipulated through the blackbox. 

**PLA** will take the input `x` multiply it by a weight `w`. The weight is initalized to whatever we want to choose or we can ask python to give us a random number. Later we will try to figure out how to set this weight so that it is the best possible. 
$$ x \cdot w = y$$
We are missing the y-intercept in out model. In machine learning it is refered to as the bias and it can be represented as just anoter weight. Lets represent our input and weights as vectors.  
$$
x = (x_1, 1) \\ 
w = (w_1, w_2)
$$
Here $w_1$ is the slope $m$ and $w_2$ is the bias $b$. If we computer the dot product of these two vectors, we will get the equation of our line as above: 
$$
   x \cdot w = x_1 \cdot w_1 + 1\cdot w_2 = y
$$

The reason we used vectors is so that we can generalized this algorithm. We can have multiple. An example can be seen below: 
$$
x = (x_1, x_2, x_3, 1) \quad w = (w_1, w_2, w_3, w_4) 
$$
$$
y = x \cdot w \\ 
  = x_1 \cdot w_1 + x_2  \cdot w_2 + x_3 \cdot w_3 + 1 \cdot w_4
$$

Note that we always have biases. This all seems abstract; however, this leads to very interesting applications. (TODO give an example). This is called the feed forward algorithm. When we are predicting we are feeding the input forward.

## General Feedforward Algorithm
The feed forward algorithm works with $n-$dimentional vectors. For the Preceptron it is just the dot product. 
$$ \sum_{i=1}^{n} x_i \cdot w_i = y $$
If you were to generalize this idea of a preceptron to multiple preceptrons this idea of a dot product will hold, but there will be multiple dot prodcuts. 
What we have described above is a regression problem, but often times this algorithm is used to classify between two or more types of outputs. We will define a function `sign` which will return `-1` if the output of the dot product is below zero and `1` if the output is above zero. 
``` Python 
sign(val):
    if val>= 0:
        return -1
    return 1
```
Our output `y` now is: `sign(dot(x, w))`. 
An implementation of the feed forward algorithm is present in the `src/preceptron/preceptron.py` file. 

``` Python 
def feed_forward(inputs, weights):
    # add the bias as part of the input
    bias = 1
    dot_product = sum([xi*wi for xi,wi in zip(inputs+[bias], weights)])
    return sign(dot_product)
```

## Training the Weights
We must train the weights so that our prediction is as acurate as it can be. We must do this by finding the error in our output, update the weights based on the error and repeat until we get a high accuracy. 

It is a _ step process: 
1. find the output with the current weights, lets call it the `guess`; 
2. you have access to the expected output (lets call it the `target`), figure out the `error = target - guess`; 
3. update all the weights based on the `error`: `weight[i] = weight[i] + error * x[i] * learning_rate`
4. repeat steps 1 to 3 until you get a high accuracy.

$$error = target - preditcion \\
w_i = w_i + error \cdot x_i \cdot learning\_rate$$

The above fourmua introduces another concept. The `learning_rate` is a number from 0 to 1 and it is useful because it only updates a percentage of the weight. If the learning rate was not there is a good chace that the weights will bounce around the targets which would result in low accuracy. We want the weihts to update gradually. 

Below is an example of a trainig function that takes in a sample input and its corresponding target value and updates the weights accordingly.
``` Python 
def train(inputs, weights, target):
  # get a guess based on the input (+1 or -1)
    guess_input = feed_forward(inputs, weigths)
    
    # get the error = known answer - guess
    error = target - guess_input
    
    # adjust the weights here (the bias is inclded as one of the weights)
    new_weights = []
    lr = 0.1
    bias = 1
    for xi, wi in zip(inputs+[bias], weights):
        # change the weights based on the previous weight, the LR,
        delta_weight = error * xi * lr
        new_weights.append(weight + delta_weight)
    return new_weights
```

## Example: The OR function
- The `OR` function is defined to have output `1` if at leat one input is true, otherwise `0`. 
| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 0 |
| 0  | 1  | 1 |
| 1  | 0  | 1 | 
| 1  | 1  | 1 | 

Therefore out inputs $x = (x_1, x_2)$, our expected outputs are $y=(0,1,1,1)$. Lets trains out PLA on this data and see how it performs. 

In [1]:
# import vector object
import sys
sys.path.append('./src/vector/')
sys.path.append('./src/preceptron/')
from vector import Vector
from preceptron import Preceptron

In [2]:
data = [[0,0], [0,1], [1,0], [1,1]]
targets = [0,1,1,1]

p_or = Preceptron(2) # will generate random weights
p_or.weights

[-0.49, 0.01, 0.35]

Right now we have random weights, so lets see what the feed forward algorithm predicts if we give it each input data: 

In [3]:
for xi in data:
    print(f'{xi} => {p_or.feed_forward(xi)}')

[0, 0] => 1
[0, 1] => 1
[1, 0] => -1
[1, 1] => -1


We are geeting negative number because we design our `sign(x)` function to be as such. We can alter it so that if when we get `1` our output of `y` is `1`, and when we get `-1` from our `feed_forward(x)` algorithm we map the output to be `0`.

In [4]:
def display_output(data):
    for xi in data: 
        ff = p_or.feed_forward(xi)
        if ff == -1:
            ff = 0
        print(f'{xi} => {ff}')
display_output(data)

[0, 0] => 1
[0, 1] => 1
[1, 0] => 0
[1, 1] => 0


Now notice that the mode is not giving accurate results; thus, we have to train it. Let us train it on all the inputs. 

In [5]:
for xi, target in zip(data, targets): 
    p_or.train(xi, target)
display_output(data)

[0, 0] => 1
[0, 1] => 1
[1, 0] => 1
[1, 1] => 1


Viola! After training we get the exact function. This was an easy eaxmple, in practice this will almost never happed because there will be alot of noise in the data and you might need to train the model for hundreds and thousands of iterations. 

## The XOR Function Example
Let us try a harder function this time. The `XOR` function is defined to be `1` if the inputs are an odd number of `1`s.

| x1 | x2 | y |
|----|----|---|
| 0  | 0  | 0 |
| 0  | 1  | 1 |
| 1  | 0  | 1 | 
| 1  | 1  | 0 | 

In [6]:
# the input data is the same, but the targets are not
targets = [0,1,1,0]

# some helper functions
def train_model(targets):
    model = Preceptron(2)
    for xi, target in zip(data, targets):
        model.train(xi, target)
    return model

def display_output(data, model):
    for xi in data: 
        ff = model.feed_forward(xi)
        if ff == -1:
            ff = 0
        print(f'{xi} => {ff}')

p_xor = train_model(targets)
display_output(data, p_xor)

[0, 0] => 0
[0, 1] => 0
[1, 0] => 0
[1, 1] => 0


As we can see, this model is function is hard to model with the preceptron. What if we train on all the inputs more than once? Lets try it, maybe in our first try we got unlucky. 

In [7]:
def train_model(targets):
    model = Preceptron(2)
    for _ in range(10):
        for xi, target in zip(data, targets):
            model.train(xi, target)
    return model
p_xor = train_model(targets)
display_output(data, p_xor)

[0, 0] => 1
[0, 1] => 1
[1, 0] => 1
[1, 1] => 1


It seems to not work. This is because the xor function is not linearly seperable. Lets try to figure out why. Graph the xor function and try to fit a line so that all the `1`s are on one side and all the `0`s are on the other side of the line. 

| inputs  | 0 | 1 |
|---------|---|---|
| 0       | 0 | 1 |
| 1       | 1 | 0 | 

The graph in show in the table above. Try to seperate this data by using only a line. It is impossible. Although the **PLA** cannot solve this problem, the generalized **PLA** (a neural network) can. 